In [1]:
import numpy as np

Améliorations possibles : 
- Prendre un autre représentant que le centroïde du cluster
- Faire un clustering différent
- Trouver des méthodes pour améliorer la complexité

Trucs à faire : 
- Comparaison avec des algos vus en cours (matrix profile, pattern decomposition inverse)
- fine tuning des paramètres ($l_M$ par exemple)